# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

c:\Users\markd\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA") #load PRICE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True) #create list of paths

dd_px = dd.read_parquet(parquet_files).set_index("ticker") # Read the parquet files into a Dask DataFrame (indexed by stock ticker) for subsequent steps



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
dd_features = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1), 
                       Adj_Close_lag_1 = x['Adj Close'].shift(1),
                       Returns = x['Close'] / x['Close'].shift(1) - 1,
                       Hi_Lo_Range = x['High'] - x['Low'], 
                       )
)

dd_features.head()  # Display the first few rows of the DataFrame with the new features



C:\Users\markd\AppData\Local\Temp\ipykernel_31020\578068986.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_features = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range
ticker,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.29
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.25
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.46
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.27
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.30


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
#compute the Dask DataFrame to a Pandas DataFrame

pd_features = dd_features.compute() 
# Calculate the rolling average of Returns over a 10-day window 
pd_features['Returns_Rolling_Avg10'] = pd_features.groupby('ticker')['Returns'].transform(lambda x: x.rolling(10, min_periods=1).mean()) #opted to allow for a minimum of 1 day of data to compute the average assuming the first day in the data set is the date of stock purchase
pd_features.head()  # Display the first few rows of the Pandas DataFrame with the new rolling average feature


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range,Returns_Rolling_Avg10
ticker,,,,,,,,,,,,,,
ACN,2016-01-04,102.620003,102.650002,100.970001,101.830002,94.543030,2817000.0,ACN.csv,2016,NaN,NaN,NaN,1.680000,NaN
ACN,2016-01-05,101.970001,102.870003,101.470001,102.360001,95.035095,2409000.0,ACN.csv,2016,101.830002,94.543030,0.005205,1.400002,0.005205
ACN,2016-01-06,100.809998,103.059998,100.540001,102.160004,94.849411,3134200.0,ACN.csv,2016,102.360001,95.035095,-0.001954,2.519997,0.001625
ACN,2016-01-07,99.750000,100.839996,98.839996,99.160004,92.064087,3194700.0,ACN.csv,2016,102.160004,94.849411,-0.029366,2.000000,-0.008705
ACN,2016-01-08,99.480003,99.809998,98.000000,98.199997,91.172775,2330200.0,ACN.csv,2016,99.160004,92.064087,-0.009681,1.809998,-0.008949


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [6]:
#No, it's not necessary to convert to Pandas just to compute a moving average. 
#However because the Dask DataFrame would require grouping by ticker and then applying a rolling function over date within ticker, it might be more efficient to compute in Pandas for a small dataset like this.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.